# Tried Dask but it has some performance isssues

Note `n_workers` and `memory_limit` must be balanced with `blocksize` to prevent the server from crashing.

```python
from dask.distributed import Client, progress, LocalCluster
import dask.dataframe as dd

worker_kwargs = {
#     'memory_limit': '20G',
#     'memory_target_fraction': 0.6,
#     'memory_spill_fraction': 0.7,
#     'memory_pause_fraction': 0.8,
    'memory_terminate_fraction': 0.98,
}

# cluster = LocalCluster(ip='0.0.0.0', dashboard_address='0.0.0.0:8788', n_workers=10, threads_per_worker=4, memory_limit='25GB', **worker_kwargs)
cluster = LocalCluster(ip='0.0.0.0', dashboard_address='0.0.0.0:8788', n_workers=30, threads_per_worker=3, memory_limit='10GB')  # , **worker_kwargs)
# cluster = LocalCluster(ip='0.0.0.0', n_workers=40, **worker_kwargs)
client = Client(cluster)

# cluster.close()
# client.close()

%%time
blocksize = int(1e8)  # 100MB

df = dd.read_json('../data/ipython_notebooks_python_and_R_scripts_imports_full/ipython_notebooks_python_and_R_scripts_imports_full-*.json', blocksize=blocksize)  # , typ='series')

print(df.npartitions)

# df.memory_usage().sum().compute()
# df = df.repartition(partition_size="100MB")
# df.npartitions

%%time
script_counts = df.drop_duplicates(subset=['repo_name', 'script_type']).script_type.value_counts()
script_counts = script_counts.compute()

# Python     562209
# IPython     23776
# R           20756
# Name: script_type, dtype: int64
```

# Load parquet dump of the data from spark

Note that we can do all this processing in spark as well.

In [1]:
import pandas as pd

In [3]:
%%time
df = pd.read_parquet('../pyspark-processing/ipython_notebooks_python_and_R_scripts_imports_full.parquet')

/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


CPU times: user 1min 56s, sys: 57.5 s, total: 2min 53s
Wall time: 1min 51s


In [6]:
%%time
vc = df.drop_duplicates(['repo_name', 'script_type'])['script_type'].value_counts()

CPU times: user 24.2 s, sys: 14 s, total: 38.2 s
Wall time: 38.1 s


In [7]:
vc

Python     562209
IPython     23776
R           20756
Name: script_type, dtype: int64

# Load library used per repo and script_type

In [2]:
%%time
mdf = pd.read_csv('../data/module_usage_per_repository/module_usage_per_repository_20191120233530.csv.gz')
py_mdf = mdf[mdf.script_type.isin(['Python', 'IPyton'])]
r_mdf = mdf[mdf.script_type.isin(['R'])]

CPU times: user 7.43 s, sys: 593 ms, total: 8.02 s
Wall time: 8.03 s


In [4]:
mdf.repo_name.nunique()

582589

In [86]:
%%time
py_module_freq = py_mdf.module.value_counts()
py_module_freq = py_module_freq[py_module_freq >= 200]

py_module_docs = py_mdf[py_mdf['module'].isin(py_module_freq.index)].groupby(['script_type', 'repo_name']).apply(lambda x: x['module'].dropna().tolist())

CPU times: user 2min 50s, sys: 4.38 s, total: 2min 54s
Wall time: 2min 52s


In [87]:
%%time
r_module_freq = r_mdf.module.value_counts()
r_module_freq = r_module_freq[r_module_freq >= 5]

r_module_docs = r_mdf[r_mdf['module'].isin(r_module_freq.index)].groupby(['script_type', 'repo_name']).apply(lambda x: x['module'].dropna().tolist())

CPU times: user 5.64 s, sys: 154 ms, total: 5.8 s
Wall time: 5.7 s


In [88]:
# module_docs = mdf.groupby(['script_type', 'repo_name']).apply(lambda x: x['module'].dropna().tolist())

In [89]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import os

def prepare_data_for_tf_projector(corpus, script_types, save_data=False, data_dir=None, min_count=100, iter=20):
    print(f'Preparing data for {len(corpus)} samples...')
    script_type = script_types[0]
    
    w2v = Word2Vec(corpus, min_count=min_count, window=100, workers=40, iter=iter, size=300, seed=1029)
    cluster = KMeans(n_clusters=10, random_state=1029)

    modules = pd.DataFrame(list(w2v.wv.vocab), columns=['module'])
    modules['vec'] = list(w2v.wv[modules.module])
    modules['cluster'] = cluster.fit_predict(list(modules['vec'].values))

    if save_data:
        assert(data_dir is not None)
        pd.DataFrame(list(modules['vec'])).to_csv(os.path.join(data_dir, f'{script_type}_modules_vec.tsv'), sep='\t', index=False, header=False)
        modules[['module', 'cluster']].to_csv(os.path.join(data_dir, f'{script_type}_modules_meta.tsv'), sep='\t', index=False, header=True)

    return modules

In [23]:
%%time
prepare_data_for_tf_projector(list(r_module_docs.loc['R'].values), script_types=['R'], save_data=True, data_dir='./', min_count=5)

Preparing data for 20597 samples...
CPU times: user 26.7 s, sys: 38.9 s, total: 1min 5s
Wall time: 3.84 s


,module,vec,cluster
0,chron,"[-0.29734376, -0.31988722, -0.17052056, -0.275...",1
1,ggplot2,"[-0.16877082, 0.11727307, -0.37764275, 0.82982...",6
2,plyr,"[-0.58273154, -0.32164478, 0.18876436, 0.55030...",6
3,grid,"[0.538545, -0.6895409, 0.0040034447, -1.374176...",6
4,rdyncall,"[-0.0015377079, -0.0013925949, -0.000474769, 0...",6
...,...,...,...
1805,PBD,"[-0.077392325, 0.13648748, -0.009825443, -0.12...",6
1806,systemPipeR,"[0.04767672, -0.01599026, 0.007497875, 0.00375...",6
1807,WMUtils,"[-0.055064663, 0.06726393, -0.020275181, 0.035...",6
1808,uaparser,"[-0.01951466, -0.017022422, -0.046944134, 0.03...",6


In [90]:
%%time
py_modules_vec = prepare_data_for_tf_projector(list(py_module_docs.loc[['Ipython', 'Python']].values), script_types=['Python'], save_data=True, data_dir='./', min_count=100, iter=10)
py_modules_vec

Preparing data for 560360 samples...
CPU times: user 2h 3min 20s, sys: 1min 56s, total: 2h 5min 16s
Wall time: 3min 14s


,module,vec,cluster
0,urlparse,"[-2.280014, -2.691734, 1.6157978, 2.1966894, -...",5
1,functools,"[4.53562, 1.0246713, -2.6429727, -0.38584536, ...",5
2,StringIO,"[0.98056847, 1.9096088, -2.8764014, -1.5552895...",8
3,copy,"[-0.45058998, -1.7555252, -1.6424855, 1.483904...",8
4,codecs,"[-1.3617383, 3.8742113, -0.2960362, -4.605664,...",8
...,...,...,...
3711,gnomedesktop,"[-0.3768868, 1.3325734, -0.0909447, 0.5488758,...",8
3712,ns3modulescan,"[-0.3964863, 0.65070814, -0.16312076, 0.720549...",8
3713,gazpacho,"[-0.8132823, 1.053119, 0.028597878, 0.80987865...",8
3714,svgitem,"[-0.29838303, 1.3231926, -0.09869748, 0.182885...",8


In [38]:
py_modules.shape

(6391, 3)

In [34]:
%%time
prepare_data_for_tf_projector(list(module_docs.loc[['Ipython', 'Python']].values), script_types=['Python'], save_data=True, data_dir='./', min_count=100)

Preparing data for 562197 samples...
CPU times: user 3h 58min 52s, sys: 4min 37s, total: 4h 3min 30s
Wall time: 7min 47s


,module,vec,cluster
0,urlparse,"[3.3688974, -5.095426, 1.7095687, 8.289695, -1...",1
1,functools,"[5.9889603, 3.6559272, 5.824282, 5.0220613, 3....",1
2,StringIO,"[0.7471799, 0.88335365, -0.63738906, 2.1189399...",1
3,copy,"[0.19896787, -4.796321, 1.2551454, 4.7062955, ...",1
4,codecs,"[2.3026829, 6.424887, 5.8550344, 4.981002, 0.6...",9
...,...,...,...
61749,geocamTrack,"[0.593021, -0.17838188, 1.0836339, 0.05259446,...",2
61750,xgds_notes2,"[0.5678476, -0.28149605, 1.1268666, 0.10522416...",2
61751,geocamUtil,"[0.6720177, -0.98314375, 1.0923384, 0.22954203...",8
61752,fondasms,"[0.04158452, -0.27844498, -0.4722933, 0.144264...",8


In [35]:
module_docs.loc[['Ipython', 'Python']]

script_type  repo_name                       
Python       0--key/lib                          [urlparse, functools, StringIO, copy, codecs, ...
             0--key/org-pub                                    [spynner, time, operator, unittest]
             0-0-1/cogbot                        [asyncio, json, typing, traceback, urllib, log...
             0-1-0/Python-Arduino-Command-API    [logging, platform, unittest, glob, Arduino, s...
             0-1-0/lightblue-0.4                 [_lightblue, datetime, _macutil, time, AppKit,...
                                                                       ...                        
             zzzsss/parsing2                       [random, threading, re, os, time, control_grid]
             zzzzRuby/Skuld                      [math, subprocess, __future__, os, docutils, u...
             zzzzrrr/openmelee                   [callbacks, xml, players, pygame, melee, gzip,...
             zzzzzsh/pyspider                    [sqlite3, pyqu

In [41]:
module_docs.loc[['Ipython', 'Python']].head()

script_type  repo_name                       
Python       0--key/lib                          [urlparse, functools, StringIO, copy, codecs, ...
             0--key/org-pub                                    [spynner, time, operator, unittest]
             0-0-1/cogbot                        [asyncio, json, typing, traceback, urllib, log...
             0-1-0/Python-Arduino-Command-API    [logging, platform, unittest, glob, Arduino, s...
             0-1-0/lightblue-0.4                 [_lightblue, datetime, _macutil, time, AppKit,...
dtype: object

In [67]:
py_module_freq['keras']

3116

12037

In [55]:
%matplotlib inline

In [65]:
module_freq.index[module_freq.index.str.contains('\.')]

Index(['data.table', 'rpart.plot', 'R.utils', 'as.character', 'org.Hs.eg.db',
       'GO.db', 'BSgenome.Hsapiens.UCSC.hg19',
       'TxDb.Hsapiens.UCSC.hg19.knownGene', 'R.matlab', 'lib.loc',
       ...
       'bioc2016.visrseq', 'BSgenome.Celegans.UCSC.ce2', 'Japan.useR',
       'env.tools', 'glm.interaction', 'SNPlocs.Hsapiens.dbSNP141.GRCh38',
       'MEF2C.data', 'RnBeads.hg19', 'install.load',
       'IlluminaHumanMethylationEPICanno.ilm10b2.hg19'],
      dtype='object', length=466)